
# Graph RAG — NL → Cypher Agent (Read‑Only)

This notebook turns **plain English** questions into **read‑only Cypher**, runs them on your Neo4j graph, and returns:
- the **raw table** (for auditability),
- a **human‑readable summary** .


In [19]:

import os, re, json, sys
from pathlib import Path
import pandas as pd
from neo4j import GraphDatabase

# Load config from project root (assumes this notebook is inside notebooks/)
repo_root = Path("..").resolve()
sys.path.append(str(repo_root))
import config

NEO4J_URI = config.NEO4J_URI
NEO4J_USER = config.NEO4J_USER
NEO4J_PASSWORD = config.NEO4J_PASSWORD

OPENAI_API_KEY = getattr(config, "OPENAI_API_KEY", None) or os.getenv("OPENAI_API_KEY")

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
driver.verify_connectivity()
print("Connected to Neo4j ✅  |  OpenAI key loaded:", bool(OPENAI_API_KEY))


Connected to Neo4j ✅  |  OpenAI key loaded: True


### Extremeley important step - adding restrictions in the building of cypher ensuring things will work out

In [20]:

ALLOWED_LABELS = {"Player","Event","Game"}
ALLOWED_RELS   = {"IN_GAME","NEXT","PERFORMED"}
WRITE_KEYWORDS = {"CREATE","MERGE","DELETE","DETACH","SET","REMOVE","CALL db.","USE","DROP","GRANT","DENY","REVOKE","LOAD CSV","APOC",":auto"}

import re

FORBIDDEN_WRITE = re.compile(
    r'(?i)\b('
    r'create|merge|delete|detach|set|remove|drop|alter|truncate|'
    r'grant|revoke|load\s+csv|import|call\s+dbms|apoc\.periodic|apoc\.load|apoc\.import|schema|index|constraint'
    r')\b'
)

def is_read_only_cypher(q: str) -> tuple[bool, str]:
    """
    Allow standard read-only Cypher with typical punctuation.
    Block known write/proc keywords, comments, multi-statements, and proc calls (except subquery CALL { ... }).
    """
    if q is None:
        return False, "Empty query."

    q_strip = q.strip()

    # single statement only
    if ";" in q_strip:
        return False, "Multiple statements not allowed."

    # block comments (avoid injection)
    if re.search(r'//|/\*|\*/', q_strip):
        return False, "Comments not allowed."

    # block dangerous keywords
    if FORBIDDEN_WRITE.search(q_strip):
        return False, "Contains forbidden write/proc keyword."

    # allow subquery `CALL { ... }`, but block procedure calls like `CALL db.*` or `CALL apoc.*`
    if re.search(r'(?i)\bcall\b', q_strip):
        # allowed: CALL { ... }
        if not re.search(r'(?i)\bcall\s*\{', q_strip):
            return False, "Procedure calls not allowed (except subquery CALL { ... })."

    # otherwise OK
    return True, "OK"

    
def cypher_df(query: str, params: dict | None = None) -> pd.DataFrame:
    ok, msg = is_read_only_cypher(query)
    if not ok:
        raise ValueError(f"Rejected non read-only Cypher: {msg}")
    with driver.session() as session:
        data = session.run(query, params or {}).data()
    return pd.DataFrame(data or [])


In [21]:

# LLM client (optional). We use the modern OpenAI SDK.
client = None
if OPENAI_API_KEY:
    try:
        from openai import OpenAI
        client = OpenAI(api_key=OPENAI_API_KEY)
    except Exception as e:
        print("OpenAI import failed; will use template fallback.", e)

SCHEMA_HINT = """
Graph schema (labels, key properties, relationships):
(:Player {player_id, name})
(:Event  {event_id, period, seconds_left_period, seconds_left_game, event_type, event_action, score, score_margin, home_desc, visit_desc, is_clutch})
(:Game   {game_id})
(:Player)-[:PERFORMED {role}]->(:Event)
(:Event)-[:IN_GAME]->(:Game)
(:Event)-[:NEXT]->(:Event)
Rules:
- Read-only queries only. Do NOT use CREATE/MERGE/SET/DELETE/REMOVE.
- Prefer returning compact tables with clear columns.
- For assists, 'role' is usually 'PLAYER2_ID'; scorer is 'PLAYER1_ID'.
- Filter clutch with e.is_clutch = true.
- Made shot has e.event_type = 1; rebound = 4.
- Order by seconds_left_period ascending or descending depending on intent.
"""

FEW_SHOT = [
    {
        "user": "Show clutch shots by Bulls role players that Michael Jordan assisted.",
        "cypher": '''
MATCH (e:Event {is_clutch: true, event_type: 1})-[:IN_GAME]->(g:Game)
MATCH (assister:Player {name: "Michael Jordan"})-[:PERFORMED]->(e)
MATCH (scorer:Player)-[:PERFORMED {role:"PLAYER1_ID"}]->(e)
WHERE scorer.name <> "Michael Jordan"
  AND EXISTS {
    MATCH (assister)-[r:PERFORMED]->(e)
    WHERE r.role IN ["PLAYER2_ID","PLAYER3_ID"]
  }
RETURN g.game_id AS game, scorer.name AS scorer, assister.name AS assister,
       e.period AS period, e.seconds_left_period AS sec_left, e.score AS score, e.score_margin AS margin,
       coalesce(e.home_desc, e.visit_desc) AS desc
ORDER BY game, sec_left
'''.strip()
    },
    {
        "user": "Give me the last 30 seconds sequence of the decisive game.",
        "cypher": '''
MATCH (e:Event {is_clutch:true})-[:IN_GAME]->(g:Game)
WITH g, count(e) AS c ORDER BY c DESC LIMIT 1
MATCH (e:Event {is_clutch:true})-[:IN_GAME]->(g)
WITH e ORDER BY e.seconds_left_period DESC LIMIT 1
MATCH p = (e)-[:NEXT*0..12]->(x:Event)
WHERE x.seconds_left_period <= e.seconds_left_period
RETURN x.event_id AS event_id, x.period AS period, x.seconds_left_period AS sec_left,
       x.event_type AS type, coalesce(x.home_desc,x.visit_desc) AS desc, x.score AS score, x.score_margin AS margin
ORDER BY period DESC, sec_left
'''.strip()
    },
    {
        "user": "Find possessions where a defensive rebound led to a go-ahead basket in the final minute.",
        "cypher": '''
MATCH (r:Event {event_type:4})-[:NEXT]->(m:Event {event_type:1, is_clutch:true})
MATCH (r)-[:IN_GAME]->(g:Game)
RETURN g.game_id AS game, r.period AS period, r.seconds_left_period AS sec_left_before,
       coalesce(r.home_desc,r.visit_desc) AS rebound_desc,
       m.seconds_left_period AS sec_left_shot, coalesce(m.home_desc,m.visit_desc) AS make_desc,
       m.score AS score_after, m.score_margin AS margin_after
ORDER BY game, sec_left_shot
LIMIT 20
'''.strip()
    }
]

def llm_to_cypher(question: str) -> str:
    # If no client, try template routing
    if client is None:
        return route_template(question)
    prompt = [
        {"role":"system","content": "You translate NL questions into READ-ONLY Cypher for the given schema. Return only the Cypher code."},
        {"role":"user","content": SCHEMA_HINT},
        {"role":"user","content": "Question: " + question},
        {"role":"user","content": "Examples:\n" + "\n\n".join([f"Q: {e['user']}\n{e['cypher']}" for e in FEW_SHOT])},
    ]
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=prompt,
        temperature=0.1,
    )
    text = resp.choices[0].message.content.strip()
    # Extract code block if present
    m = re.search(r"```(?:cypher)?\s*(?P<q>.+?)```", text, flags=re.S)
    cypher = m.group("q").strip() if m else text
    ok, msg = is_read_only_cypher(cypher)
    if not ok:
        raise ValueError(f"LLM produced non read-only Cypher: {msg}\n---\n{cypher}")
    return cypher

def route_template(question: str) -> str:
    q = question.lower()
    if "assist" in q and ("clutch" in q or "last" in q):
        return FEW_SHOT[0]["cypher"]
    if ("last 30" in q or "final 30" in q or "last thirty" in q) and ("sequence" in q or "timeline" in q):
        return FEW_SHOT[1]["cypher"]
    if ("rebound" in q and ("go-ahead" in q or "lead" in q) and ("final minute" in q or "last minute" in q or "clutch" in q)):
        return FEW_SHOT[2]["cypher"]
    # default safe query: top clutch makes
    return '''
MATCH (e:Event {is_clutch:true, event_type:1})-[:IN_GAME]->(g:Game)
OPTIONAL MATCH (scorer:Player)-[:PERFORMED {role:"PLAYER1_ID"}]->(e)
OPTIONAL MATCH (assist:Player)-[:PERFORMED {role:"PLAYER2_ID"}]->(e)
RETURN g.game_id AS game, e.period AS period, e.seconds_left_period AS sec_left,
       scorer.name AS scorer, assist.name AS assister,
       e.score AS score, e.score_margin AS margin, coalesce(e.home_desc,e.visit_desc) AS desc
ORDER BY game, sec_left
'''


In [22]:

def summarize_answer(question: str, df: pd.DataFrame) -> str:
    if df.empty:
        return f"I ran a read-only graph query for: '{question}'. No rows matched."
    # If LLM available, generate crisp summary using few lines of the DataFrame
    if client is not None:
        sample = df.head(10).to_dict(orient="records")
        prompt = [
            {"role":"system","content":"You are a precise sports analyst. Summarize the table as 2–5 bullet points. No hype."},
            {"role":"user","content": f"Question: {question}"},
            {"role":"user","content": "Rows (JSON): " + json.dumps(sample)},
        ]
        try:
            resp = client.chat.completions.create(model="gpt-4o-mini", messages=prompt, temperature=0.2)
            return resp.choices[0].message.content.strip()
        except Exception as e:
            pass
    # Fallback: simple heuristic bullets
    cols = df.columns.tolist()
    lines = []
    lines.append(f"• Returned {len(df)} rows. Showing first {min(10, len(df))}.")
    key_cols = [c for c in ["game","scorer","assister","period","sec_left","margin"] if c in cols]
    if key_cols:
        lines.append("• Key columns: " + ", ".join(key_cols))
    if "scorer" in cols:
        top_scorers = df["scorer"].value_counts().head(3).to_dict()
        lines.append("• Top scorers in results: " + ", ".join([f"{k} ({v})" for k,v in top_scorers.items()]))
    if "assister" in cols:
        top_ast = df["assister"].value_counts().head(3).to_dict()
        lines.append("• Top assisters: " + ", ".join([f"{k} ({v})" for k,v in top_ast.items()]))
    if "sec_left" in cols:
        sec_min = df["sec_left"].min()
        sec_max = df["sec_left"].max()
        lines.append(f"• Seconds-left range in results: {sec_min}–{sec_max}.")
    return "\n".join(lines)


In [23]:

def ask(question: str, verbose: bool = True) -> dict:
    cypher = llm_to_cypher(question)
    if verbose:
        print("CYTHER (read-only):\n", cypher)
    df = cypher_df(cypher)
    summary = summarize_answer(question, df)
    return {"cypher": cypher, "df": df, "summary": summary}


## Try it

In [24]:

res = ask("Show me clutch shots by Bulls role players that Michael Jordan assisted.")
display(res["df"].head(20))
print()
print(res["summary"])


CYTHER (read-only):
 MATCH (e:Event {is_clutch: true, event_type: 1})-[:IN_GAME]->(g:Game)
MATCH (assister:Player {name: "Michael Jordan"})-[:PERFORMED]->(e)
MATCH (scorer:Player)-[:PERFORMED {role:"PLAYER1_ID"}]->(e)
WHERE scorer.name <> "Michael Jordan"
  AND EXISTS {
    MATCH (scorer)-[:PERFORMED {role:"PLAYER2_ID"}]->(e)
  }
RETURN g.game_id AS game, scorer.name AS scorer, assister.name AS assister,
       e.period AS period, e.seconds_left_period AS sec_left, e.score AS score, e.score_margin AS margin,
       coalesce(e.home_desc, e.visit_desc) AS desc
ORDER BY g.game_id, sec_left


""



I ran a read-only graph query for: 'Show me clutch shots by Bulls role players that Michael Jordan assisted.'. No rows matched.


In [31]:
res = ask("Give me the last 30 game seconds when someone scored and provided a last moment victory.")
display(res["df"].head(30))
print()
print(res["summary"])


CYTHER (read-only):
 MATCH (e:Event {is_clutch: true, event_type: 1})-[:IN_GAME]->(g:Game)
WHERE e.seconds_left_game <= 30
RETURN g.game_id AS game, e.period AS period, e.seconds_left_period AS sec_left,
       e.score AS score, e.score_margin AS margin,
       coalesce(e.home_desc, e.visit_desc) AS desc
ORDER BY e.seconds_left_period ASC


,game,period,sec_left,score,margin,desc
0,49600083,4,0,82 - 84,2.0,Jordan 19' Jump Shot (31 PTS)
1,49600088,4,5,86 - 88,2.0,Kerr 14' Jump Shot (9 PTS) (Jordan 4 AST)
2,49600087,4,6,90 - 87,-3.0,NaN
3,49600087,4,15,88 - 87,-1.0,Ostertag Layup (13 PTS) (Stockton 5 AST)
4,49600087,4,25,88 - 85,-3.0,NaN



- In game 49600083, Jordan scored a jump shot with no time left, resulting in a final score of 82-84.
- In game 49600088, Kerr made a jump shot with 5 seconds remaining, bringing the score to 86-88.
- In game 49600087, Ostertag scored a layup with 15 seconds left, making the score 88-87.
- The last moments of game 49600087 also included a score of 90-87, but no specific scoring event was noted.


In [25]:
res = ask("Give me the last 30 seconds sequence of the Kerr moment decisive game.")
display(res["df"].head(30))
print()
print(res["summary"])


CYTHER (read-only):
 MATCH (e:Event {is_clutch:true})-[:IN_GAME]->(g:Game)
WITH g, count(e) AS c ORDER BY c DESC LIMIT 1
MATCH (e:Event {is_clutch:true})-[:IN_GAME]->(g)
WITH e ORDER BY e.seconds_left_period DESC LIMIT 1
MATCH p = (e)-[:NEXT*0..12]->(x:Event)
WHERE x.seconds_left_period <= e.seconds_left_period AND x.seconds_left_period <= 30
RETURN x.event_id AS event_id, x.period AS period, x.seconds_left_period AS sec_left,
       x.event_type AS type, coalesce(x.home_desc,x.visit_desc) AS desc, x.score AS score, x.score_margin AS margin
ORDER BY period DESC, sec_left


,event_id,period,sec_left,type,desc,score,margin
0,49600083_398,4,7,4,Jordan REBOUND (Off:1 Def:3),NaN,-1.0
1,49600083_399,4,7,9,BULLS Timeout: Regular (Full 5 Short 1),NaN,-1.0
2,49600083_401,4,7,8,SUB: Kerr FOR Rodman,NaN,-1.0
3,49600083_402,4,7,8,SUB: Kukoc FOR Longley,NaN,-1.0
4,49600083_403,4,7,8,SUB: Buechler FOR Harper,NaN,-1.0
5,49600083_391,4,9,4,NaN,NaN,-1.0
6,49600083_392,4,9,6,Rodman L.B.FOUL (P4.PN),NaN,-1.0
7,49600083_393,4,9,3,NaN,NaN,-1.0
8,49600083_394,4,9,4,NaN,NaN,-1.0
9,49600083_395,4,9,8,SUB: Longley FOR Kukoc,NaN,-1.0



- With 9 seconds left in the game, Rodman committed a foul, leading to a stoppage in play.
- The Bulls called a timeout to strategize, with 7 seconds remaining.
- Substitutions were made: Kerr replaced Rodman, Kukoc replaced Longley, and Buechler replaced Harper.
- Jordan secured a rebound with 7 seconds left, maintaining a one-point deficit.
- The game continued with the Bulls down by one point, indicating a critical moment in the final seconds.


In [26]:

res = ask("Find possessions where a defensive rebound led immediately to a go-ahead basket in the final minute.")
display(res["df"].head(20))
print()
print(res["summary"])


CYTHER (read-only):
 MATCH (r:Event {event_type: 4})-[:NEXT]->(m:Event {event_type: 1, is_clutch: true})
MATCH (r)-[:IN_GAME]->(g:Game)
WHERE r.seconds_left_period <= 60
RETURN g.game_id AS game, r.period AS period, r.seconds_left_period AS sec_left_before,
       coalesce(r.home_desc, r.visit_desc) AS rebound_desc,
       m.seconds_left_period AS sec_left_shot, coalesce(m.home_desc, m.visit_desc) AS make_desc,
       m.score AS score_after, m.score_margin AS margin_after
ORDER BY m.seconds_left_period ASC
LIMIT 20


,game,period,sec_left_before,rebound_desc,sec_left_shot,make_desc,score_after,margin_after
0,49600087,4,42,NaN,25,NaN,88 - 85,-3.0



- In game 49600087, during the 4th period with 42 seconds left, a defensive rebound occurred.
- The subsequent shot was made with 25 seconds remaining, resulting in a go-ahead basket.
- The score after this possession was 88 - 85, giving a margin of -3.0.


In [27]:
res = ask("Dad, was there a time in the Finals when Jordan passed at the very end and someone else nailed the winner? Who was it and when?")
display(res["df"].head(20))
print()
print(res["summary"])

CYTHER (read-only):
 MATCH (e:Event {is_clutch: true})-[:IN_GAME]->(g:Game)
MATCH (passer:Player)-[:PERFORMED {role:"PLAYER2_ID"}]->(e)
MATCH (scorer:Player)-[:PERFORMED {role:"PLAYER1_ID"}]->(e)
WHERE passer.name = "Michael Jordan"
RETURN g.game_id AS game, scorer.name AS scorer, e.period AS period, 
       e.seconds_left_period AS sec_left, e.score AS score, e.score_margin AS margin,
       coalesce(e.home_desc, e.visit_desc) AS desc
ORDER BY e.seconds_left_period DESC
LIMIT 1


,game,scorer,period,sec_left,score,margin,desc
0,49600088,Steve Kerr,4,5,86 - 88,2.0,Kerr 14' Jump Shot (9 PTS) (Jordan 4 AST)



- In Game 5 of the Finals, Michael Jordan passed the ball to Steve Kerr.
- Kerr made a jump shot with 5 seconds left in the 4th period.
- The score at the time of the shot was 86-88, with the margin being 2 points. 
- This was Kerr's 9th point of the game, and it was assisted by Jordan, who had 4 assists in that game.


In [28]:
res = ask("Dad, was the best clutch moment of all the games?")
display(res["df"].head(20))
print()
print(res["summary"])

CYTHER (read-only):
 MATCH (e:Event {is_clutch: true})-[:IN_GAME]->(g:Game)
RETURN g.game_id AS game, e.period AS period, e.seconds_left_period AS sec_left,
       e.event_type AS type, coalesce(e.home_desc, e.visit_desc) AS desc, e.score AS score, e.score_margin AS margin
ORDER BY sec_left


,game,period,sec_left,type,desc,score,margin
0,49600083,4,0,1,Jordan 19' Jump Shot (31 PTS),82 - 84,2.0
1,49600083,4,0,13,NaN,82 - 84,2.0
2,49600087,4,0,4,Stockton REBOUND (Off:2 Def:1),NaN,-3.0
3,49600087,4,0,8,NaN,NaN,-3.0
4,49600087,4,0,8,SUB: Ostertag FOR Carr,NaN,-3.0
5,49600087,4,0,6,NaN,NaN,-3.0
6,49600087,4,0,3,MISS Stockton Free Throw 1 of 2,NaN,-3.0
7,49600087,4,0,4,JAZZ Rebound,NaN,-3.0
8,49600087,4,0,3,Stockton Free Throw 2 of 2 (13 PTS),90 - 88,-2.0
9,49600087,4,0,13,NaN,90 - 88,-2.0



- In Game 49600083, Michael Jordan made a crucial jump shot with no time left, bringing his total to 31 points and narrowing the score to 82-84.
- In Game 49600087, John Stockton missed his first free throw but successfully made the second, scoring 13 points overall and finalizing the score at 90-88.
- Both games featured significant moments in the final seconds, impacting the score margins directly.


In [29]:
res = ask("Show the most decisive clutch field goal: a lead-changing shot in the last 30 seconds after which the lead never flipped again.")
display(res["df"].head(20))
print()
print(res["summary"])

CYTHER (read-only):
 MATCH (e:Event {is_clutch: true, event_type: 1})-[:IN_GAME]->(g:Game)
MATCH (scorer:Player)-[:PERFORMED {role:"PLAYER1_ID"}]->(e)
WHERE e.seconds_left_period <= 30
WITH e, g, scorer
ORDER BY e.seconds_left_period ASC
WITH e, g, scorer, e.score_margin AS margin
MATCH (next:Event)-[:NEXT]->(e)
WHERE next.score_margin <> margin
RETURN g.game_id AS game, scorer.name AS scorer, e.period AS period, 
       e.seconds_left_period AS sec_left, e.score AS score, 
       e.score_margin AS margin, coalesce(e.home_desc, e.visit_desc) AS desc
LIMIT 1


,game,scorer,period,sec_left,score,margin,desc
0,49600083,Michael Jordan,4,0,82 - 84,2.0,Jordan 19' Jump Shot (31 PTS)



- The decisive clutch field goal was made by Michael Jordan.
- The shot occurred in the 4th period with 0 seconds left on the clock.
- The score before the shot was 82 - 84, giving Jordan's team a 2-point lead after the basket.
- The shot was a 19-foot jump shot, contributing to Jordan's total of 31 points in the game.
- After this shot, the lead did not change again.


In [49]:
res = ask("Who made the clutch shot that gave the Bulls their final lead in Game 6 of the 1997 NBA Finals?")
display(res["df"].head(30))
print()
print(res["summary"])


CYTHER (read-only):
 MATCH (e:Event {is_clutch: true, event_type: 1})-[:IN_GAME]->(g:Game {game_id: "1997_G6"})
MATCH (scorer:Player)-[:PERFORMED {role:"PLAYER1_ID"}]->(e)
RETURN g.game_id AS game, scorer.name AS scorer, e.period AS period, e.seconds_left_period AS sec_left,
       e.score AS score, e.score_margin AS margin, coalesce(e.home_desc, e.visit_desc) AS desc
ORDER BY e.seconds_left_period ASC
LIMIT 1


""



I ran a read-only graph query for: 'Who made the clutch shot that gave the Bulls their final lead in Game 6 of the 1997 NBA Finals?'. No rows matched.


In [ ]:

driver.close()
print("Done. ✅")
